%%latex
\tableofcontents

In [11]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 300
import random
import csv
import h5py
import pandas as pd
import pickle
import torch
# import torchvision
from torch import nn
from torch.utils.data import Dataset, DataLoader
import matplotlib.cm as cm
import pickle
import os
# Own modules:
import physics
import data
from data import CustomDataset
import nnc2p
# Get master directory
master_dir = os.path.normpath(os.path.join(os.getcwd(), '..'))
print(master_dir)

D:\Thibeau\master-thesis-AI


# Introduction

This notebook is a complete mess, so please don't look at it, this is just my playground to quickly test a few nets, and export hem etc.

# Architectures

In [3]:
class FeedForwardNetwork(nn.Module):
    """
    Implements a simple feedforward neural network.
    """
    def __init__(self, h: list = [3, 600, 200, 1], activation_function = nn.Sigmoid, output_bias=True) -> None:
        """
        Initialize the neural network class.
        """
        # Call the super constructor first
        super(FeedForwardNetwork, self).__init__()

        # For convenience, save the sizes of the hidden layers as fields as well
        self.h = h

        # Define the layers:
        for i in range(len(self.h)-1):
            if i == len(self.h)-2:
                setattr(self, f"linear{i+1}", nn.Linear(self.h[i], self.h[i+1], bias=output_bias))
            else:
                setattr(self, f"linear{i+1}", nn.Linear(self.h[i], self.h[i+1]))
                setattr(self, f"activation{i+1}", activation_function())

    def forward(self, x):
        """
        Computes a forward step given the input x.
        :param x: Input for the neural network.
        :return: x: Output neural network
        """

        for i, module in enumerate(self.modules()):
            # The first module is the whole NNC2P object, continue
            if i == 0:
                continue
            x = module(x)

        return x

In [18]:
class Net(nn.Module):
    """
    Implements a simple feedforward neural network.
    """
    def __init__(self, h: list = [3, 600, 200, 1], activation_function = nn.Sigmoid, output_bias=True) -> None:
        """
        Initialize the neural network class.
        """
        # Call the super constructor first
        super(Net, self).__init__()

        # For convenience, save the sizes of the hidden layers as fields as well
        self.h = h

        # Define the layers:
        for i in range(len(self.h)-1):
            if i == len(self.h)-2:
                setattr(self, f"linear{i+1}", nn.Linear(self.h[i], self.h[i+1], bias=output_bias))
            else:
                setattr(self, f"linear{i+1}", nn.Linear(self.h[i], self.h[i+1]))
                setattr(self, f"activation{i+1}", activation_function())

    def forward(self, x):
        """
        Computes a forward step given the input x.
        :param x: Input for the neural network.
        :return: x: Output neural network
        """

        for i, module in enumerate(self.modules()):
            # The first module is the whole NNC2P object, continue
            if i == 0:
                continue
            x = module(x)

        return x

# Update: using the ONNX format

(Semester 2) I'm exporting a model again, now in ONNX format. 

In [2]:
# Load a model
state_dict = torch.load("finetuned_most_pruned.pth")
model = nnc2p.create_nn(state_dict)
model = model.float()
model

NeuralNetwork(
  (linear1): Linear(in_features=3, out_features=504, bias=True)
  (linear2): Linear(in_features=504, out_features=127, bias=True)
  (linear3): Linear(in_features=127, out_features=1, bias=True)
)

In [3]:
torch.onnx.export(model, torch.randn(3).float(), "pruned_model.onnx")

Testing the results:

In [7]:
df = pd.read_csv("../Data/ideal_gas_c2p_test_data.csv")
df.head(1)

,rho,eps,v,p,D,S,tau
0,9.836323,1.962039,0.266066,12.866164,10.204131,12.026585,22.131297


Get a first input for the network:

In [6]:
x = np.array([df["D"][0], df["S"][0], df["tau"][0]])
x

array([10.20413115, 12.02658484, 22.13129693])

Check inference on model:

In [7]:
with torch.no_grad():
    out = model(torch.from_numpy(x).float()).numpy()
    print(out[0])

12.866582


In [29]:
model.state_dict()

OrderedDict([('linear1.weight',
              tensor([[-0.3666,  0.4540, -0.4352],
                      [ 0.0356,  0.9699,  0.4002],
                      [ 0.1087, -0.0912,  0.1072],
                      ...,
                      [ 0.5475, -0.5256, -0.2971],
                      [-0.4310, -0.1393,  0.2783],
                      [ 0.6337, -0.3134,  0.0312]])),
             ('linear1.bias',
              tensor([ 5.6061e-01,  2.4333e-01, -7.7467e-01, -3.2593e-01,  4.9073e-02,
                       7.0926e-03,  2.2542e-01, -5.3934e-01,  2.5813e-01, -3.1620e-01,
                      -4.9130e-01, -3.7876e-01, -2.5505e-01,  7.9304e-01,  7.3299e-01,
                       1.0455e-01,  8.8842e-01,  1.4683e-01, -4.3929e-01, -3.4603e-01,
                      -6.0459e-01, -7.6528e-01, -4.8161e-01, -4.2301e-01, -1.1967e-01,
                      -7.1304e-01,  2.7122e-01,  1.1723e-01,  5.4620e-01, -1.7614e-01,
                      -1.1986e-02, -3.2823e-01, -1.2451e-01, -2.4551e-01,  5.527

__NOTE__ The most recent architecture:

__NOTE__ the best & most pruned model

In [3]:
new_model = FeedForwardNetwork(h=[3, 504, 127, 1])
new_model.load_state_dict(torch.load("new_finetuned_most_pruned.pth"))

RuntimeError: Error(s) in loading state_dict for FeedForwardNetwork:
	Missing key(s) in state_dict: "linear3.bias". 

In [22]:
x = np.array([df["D"][0], df["S"][0], df["tau"][0]])
x
x = np.ones((1,3))
x

array([[1., 1., 1.]])

Check inference on model:

In [23]:
with torch.no_grad():
    out = new_model(torch.from_numpy(x).float()).numpy()
    print(out[0])

[0.4520222]


to save ENTIRE model

In [12]:
# torch.save(new_model, "new_finetuned_most_pruned.pt")

Testing PyTorch load

In [17]:
test = torch.load("new_finetuned_most_pruned.pt")
test

AttributeError: Can't get attribute 'FeedForwardNetwork' on <module '__main__'>

# Use pickle

We use pickle to save the whole neural net object

We redefine the architecture such that it can be used without nnc2p

In [14]:
# """Our network architecture"""
# class NeuralNetwork(nn.Module):
#     """
#     Implements a two-layered neural network for the C2P conversion. Note that hence the number of layers is fixed
#     for this NN subclass! The activation functions are sigmoids.
#     """

#     def __init__(self, h1: int = 600, h2: int = 200) -> None:
#         """
#         Initialize the neural network class.
#         :param name: String that names this network, in order to recognize it later on.
#         :param h1: Size (number of neurons) of the first hidden layer.
#         :param h2: Size (number of neurons) of the second hidden layer.
#         """
#         # Call the super constructor first
#         super(NeuralNetwork, self).__init__()

#         # For convenience, save the sizes of the hidden layers as fields as well
#         self.h1 = h1
#         self.h2 = h2

#         # Define the weights:
#         self.linear1 = nn.Linear(3, h1)
#         self.linear2 = nn.Linear(h1, h2)
#         self.linear3 = nn.Linear(h2, 1)

#     def forward(self, x):
#         """
#         Computes a forward step given the input x.
#         :param x: Input for the neural network.
#         :return: x: Output neural network
#         """

#         x = self.linear1(x)
#         x = torch.sigmoid(x)
#         x = self.linear2(x)
#         x = torch.sigmoid(x)
#         x = self.linear3(x)
#         return x

In [15]:
# # Two auxiliary functions are needed to load the weights from the state dict into a fresh network
# def get_hidden_sizes_from_state_dict(state_dict):
#     """
#     Finds the sizes of the two hidden layers of our 2-layer architecture given a state dict.
#     :param state_dict: State dict of saved parameters
#     :return: h1, size of first hidden layer, and h2, size of second hidden layer
#     """
#     h1 = np.shape(state_dict['linear1.bias'])[0]
#     h2 = np.shape(state_dict['linear2.bias'])[0]

#     return h1, h2

# def create_nn(state_dict):
#     """
#     Create a NeuralNetwork object if given a dictionary of the weights, with correct sizes for hidden layers.
#     :param state_dict: State dictionary containing the weights of the neural network
#     :return:
#     """
#     h1, h2 = get_hidden_sizes_from_state_dict(state_dict)
#     model = NeuralNetwork(h1=h1, h2=h2)
#     model.load_state_dict(state_dict)

#     return model

In [17]:
state_dict = torch.load("finetuned_most_pruned.pth")
model = create_nn(state_dict)
model = model.float()
model

NeuralNetwork(
  (linear1): Linear(in_features=3, out_features=504, bias=True)
  (linear2): Linear(in_features=504, out_features=127, bias=True)
  (linear3): Linear(in_features=127, out_features=1, bias=True)
)

In [18]:
# # Open a file and use dump()
# filename = "model.pkl"
# with open(filename, 'wb') as file:
      
#     # A new file will be created
#     pickle.dump(model, file)

Check: to load again:

In [19]:
# # Open the file in binary mode
# with open(filename, 'rb') as file:
      
#     # Call load method to deserialze
#     test = pickle.load(file)
  
#     print(test)

NeuralNetwork(
  (linear1): Linear(in_features=3, out_features=504, bias=True)
  (linear2): Linear(in_features=504, out_features=127, bias=True)
  (linear3): Linear(in_features=127, out_features=1, bias=True)
)


# Most recent model for tabulated EOS/NN assist

Load the tabulated EOS for (50, 50)

In [19]:
h = [3, 10, 10, 1]
model = Net(h=h, output_bias=False).float()

For neural nets assist:

In [20]:
model.load_state_dict(torch.load(os.path.join(master_dir, "Models/NNassist_10_10.pth")))

<All keys matched successfully>

Most recent architecture for NNEOS

In [8]:
# model.load_state_dict(torch.load("nn_tabeos_3_50_50_3.pth"))

<All keys matched successfully>

# (Archive) Preliminaries

This is the model architecture:

In [2]:
# Define hyperparameters of the model here. Will first of all put two hidden layers
# total of 800 neurons for the one in the paper
device = "cpu"
size_HL_1 = 600
size_HL_2 = 200

# Implement neural network
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        #self.flatten = nn.Flatten()
        self.stack = nn.Sequential(
            nn.Linear(3, size_HL_1),
            nn.Sigmoid(),
            nn.Linear(size_HL_1, size_HL_2),
            nn.Sigmoid(),
            nn.Linear(size_HL_2, 1)
        )

    def forward(self, x):
        # No flatten needed, as our input and output are 1D?
        #x = self.flatten(x) 
        logits = self.stack(x)
        return logits

We import NNC2Pv0, which was on par with the models in the paper. The t2 version is trained a bit longer than the version of the paper. Use OS to locate the Models folder correctly.

In [20]:
# # Directory of current file:
# dir_path = os.path.abspath("D:\Coding\master-thesis-AI\Code\Semester 1")
# # Models folder
# models_folder =os.path.abspath("D:\Coding\master-thesis-AI\Models")
# print("Models folder: " + models_folder)
# # Move to the models folder
# os.chdir(models_folder)

Models folder: D:\Coding\master-thesis-AI\Models


In [24]:
# Move to the correct folder
file_location = os.chdir("D:\Coding\master-thesis-AI\Models")
NNC2P = torch.load("NNC2Pv0t2.pth")
model = NNC2P

In case we want to view the variables, uncomment the following:

In [25]:
# NNC2P.state_dict()

# Save the matrices as a CSV

Note that converting from Torch tensor to Numpy array does __not__ cause loss of information:

In [29]:
# test_exact = NNC2P.state_dict()["stack.0.weight"]
# test_exact_value = test_exact[0][0].item()
# print('%.25f' % test_exact_value)
# print("---")
# test_exact_np = test_exact.numpy()
# test_exact_value_np = test_exact_np[0][0]
# print('%.25f' % test_exact_value_np)

-0.3647063672542572021484375
---
-0.3647063672542572021484375


 ## Saving and loading as CSV

Save the values: ([refresher on Pickle](https://tech.qvread.com/python/python-list-read-write-csv/))

In [21]:
dir_name = os.path.normpath("../Models/paramvals_assist//")
print(f"Saving model CSV files to {dir_name}")

Saving model CSV files to ..\Models\paramvals_assist


In [22]:
# State dict contains all the variables
state_dict = model.state_dict().items()
# Names to save the files:
file_names            = ["weight0", "bias0", "weight2", "bias2", "weight4", "bias4"]
save_names         = [os.path.normpath(dir_name + "/" + name + ".csv") for name in file_names]
flat_save_names = [os.path.normpath(dir_name + "/" + name + "_flat.csv") for name in file_names]
no_comma_flat_save_names = [os.path.normpath(dir_name + "/" + name + "_flat_no_comma.csv") for name in file_names]

# Save each one:
counter = 0
for param_name, item in state_dict:
    # Get appropriate names
    name                   = file_names[counter]
    save_name         = save_names[counter]
    flat_save_name = flat_save_names[counter]
    no_comma_flat_save_name = no_comma_flat_save_names[counter]
    # Get the matrix and flatten it as well
    matrix_np   = item.numpy() 
    flat_matrix_np   = matrix_np.flatten()
    # The following save txt is only important for stuff done within this noteboo!
    np.savetxt(no_comma_flat_save_name, flat_matrix_np, delimiter=",", fmt="%0.35f")
    
    np.savetxt(save_name, matrix_np, delimiter=",", fmt="%0.35f")
    # Note: due to weird Fortran stuff, have to append a 0 at the start of the file
    flat_matrix_np   = np.insert(flat_matrix_np, 0, 0)
    np.savetxt(flat_save_name, flat_matrix_np, delimiter=",", newline=',\n', fmt="%0.35f")
    
    counter += 1

Read the files:

In [96]:
# weight0 = np.loadtxt('Models/paramvals/weight0.csv', delimiter=",")
# bias0      = np.loadtxt('Models/paramvals/bias0.csv', delimiter=",")
# s = np.shape(bias0)[0]
# bias0 = np.reshape(bias0, (s, 1))
# weight2 = np.loadtxt('Models/paramvals/weight2.csv', delimiter=",")
# bias2      = np.loadtxt('Models/paramvals/bias2.csv', delimiter=",")
# s = np.shape(bias2)[0]
# bias2 = np.reshape(bias2, (s, 1))
# weight4 = np.loadtxt('Models/paramvals/weight4.csv', delimiter=",")
# s = np.shape(weight4)[0]
# weight4 = np.reshape(weight4, (1, s))
# bias4      = np.loadtxt('Models/paramvals/bias4.csv', delimiter=",")
# bias4 = np.reshape(bias4, (1, 1))

# weights_and_biases = [weight0, bias0, weight2, bias2, weight4, bias4]

Same for flat: __NOTE__ for numpy (here), we load "no_comma" files since otherwise there's an error. For Fortran, we use the files __WITHOUT__ "no comma".

In [97]:
# weight0_flat = np.loadtxt('Models/paramvals/weight0_flat_no_comma.csv', delimiter=",")
# bias0_flat      = np.loadtxt('Models/paramvals/bias0_flat_no_comma.csv', delimiter=",")
# weight2_flat = np.loadtxt('Models/paramvals/weight2_flat_no_comma.csv', delimiter=",")
# bias2_flat      = np.loadtxt('Models/paramvals/bias2_flat_no_comma.csv', delimiter=",")
# weight4_flat = np.loadtxt('Models/paramvals/weight4_flat_no_comma.csv', delimiter=",")
# bias4_flat      = np.loadtxt('Models/paramvals/bias4_flat_no_comma.csv', delimiter=",")

# weights_and_biases_flat = [weight0_flat, bias0_flat, weight2_flat, bias2_flat, weight4_flat, bias4_flat]

In [98]:
print('%.25f' % weight0[0][0])

-0.3647063672542572021484375


In [99]:
np.shape(weight0_flat)

(1800,)

(Below: old pickle version)

In [100]:
#   reload pickled data from file
# test_name = save_names[0]
# with open(test_name, 'rb') as f:
#     test_load = pickle.load(f)

In [101]:
# # Save the loaded versions in the appropriate variables
# with open('Models/paramvals/weight0.csv', 'rb') as f:
#     weight0 = pickle.load(f)
    
# with open('Models/paramvals/bias0.csv', 'rb') as f:
#     bias0 = pickle.load(f)
#     s = np.shape(bias0)[0]
#     bias0 = np.reshape(bias0, (s, 1))
    
# with open('Models/paramvals/weight2.csv', 'rb') as f:
#     weight2 = pickle.load(f)
    
# with open('Models/paramvals/bias2.csv', 'rb') as f:
#     bias2 = pickle.load(f)
#     s = np.shape(bias2)[0]
#     bias2 = np.reshape(bias2, (s, 1))
    
# with open('Models/paramvals/weight4.csv', 'rb') as f:
#     weight4 = pickle.load(f)
    
# with open('Models/paramvals/bias4.csv', 'rb') as f:
#     bias4 = pickle.load(f)
#     s = np.shape(bias4)[0]
#     bias4 = np.reshape(bias4, (s, 1))

# # Gather together in a list of all variables
# weights_and_biases = [weight0, bias0, weight2, bias2, weight4, bias4]

Same for flattened arrays:

In [102]:
# with open('Models/paramvals/bias0_flat.csv', 'r') as file:
#     csvreader = csv.reader(file)
# #     for row in csvreader:
# #         print(row)

In [103]:
# # Save the loaded versions in the appropriate variables
# with open('Models/paramvals/weight0_flat.csv', 'rb') as f:
#     weight0_flat = pickle.load(f)
    
# # with open('Models/paramvals/bias0_flat.csv', 'rb') as f:
# #     bias0_flat = pickle.load(f)

    
# with open('Models/paramvals/weight2_flat.csv', 'rb') as f:
#     weight2_flat = pickle.load(f)
    
# with open('Models/paramvals/bias2_flat.csv', 'rb') as f:
#     bias2_flat = pickle.load(f)
    
# with open('Models/paramvals/weight4_flat.csv', 'rb') as f:
#     weight4_flat = pickle.load(f)
    
# with open('Models/paramvals/bias4_flat.csv', 'rb') as f:
#     bias4_flat = pickle.load(f)

# # Gather together in a list of all variables
# weights_and_biases_flat = [weight0_flat, bias0_flat, weight2_flat, bias2_flat, weight4_flat, bias4_flat]

In [104]:
# Print the shape for each parameter:
for i in range(len(weights_and_biases)):
    print("For the file: ", file_names[i])
    # Read the values
    shape = np.shape(weights_and_biases[i])
    print("The shape is equal to ", shape)

For the file:  weight0
The shape is equal to  (600, 3)
For the file:  bias0
The shape is equal to  (600, 1)
For the file:  weight2
The shape is equal to  (200, 600)
For the file:  bias2
The shape is equal to  (200, 1)
For the file:  weight4
The shape is equal to  (1, 200)
For the file:  bias4
The shape is equal to  (1, 1)


In [105]:
# Same for their flattened versions:
for i in range(len(weights_and_biases_flat)):
    print("For the file: ", flat_save_names[i])
    # Read the values
    shape = np.shape(weights_and_biases_flat[i])
    print("The shape is equal to ", shape)

For the file:  Models/paramvals/weight0_flat.csv
The shape is equal to  (1800,)
For the file:  Models/paramvals/bias0_flat.csv
The shape is equal to  (600,)
For the file:  Models/paramvals/weight2_flat.csv
The shape is equal to  (120000,)
For the file:  Models/paramvals/bias2_flat.csv
The shape is equal to  (200,)
For the file:  Models/paramvals/weight4_flat.csv
The shape is equal to  (200,)
For the file:  Models/paramvals/bias4_flat.csv
The shape is equal to  ()


##### Play around with some examples

In [88]:
# # Read the example file
# print(example)
# print(np.shape(example))

In [89]:
# test_load_value = test_load[0][0]
# print('%.25f' % test_load_value)

## Predicting using the values in the arrays

When we are going to implement this in the Gmunu code, we can no longer use any of the built-in tools of PyTorch.

In [106]:
## One specific test case for the data
rho,eps,v,p,D,S,tau = 9.83632270803203,1.962038705851822,0.2660655147967911,12.866163917605371,10.204131145455385,12.026584842282125,22.131296926293793

This is how the PyTorch implementation works:

In [107]:
input_test = torch.tensor([D, S, tau])
exact_result = p
print("Exact:")
print(exact_result)
# print(input_test)
with torch.no_grad():
    pred = model(input_test).item()

print("Pytorch prediction:")
print(pred)

Exact:
12.866163917605371
Pytorch prediction:
12.866371154785156


Now, we have to try and get the same output, but by defining all intermediate steps ourselves!

In [108]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

def compute_prediction(x):
    """Input is a np. array of size 1x3"""
    x = np.matmul(weight0, x) + bias0
    x = sigmoid(x)
    x = np.matmul(weight2, x) + bias2
    x = sigmoid(x)
    x = np.matmul(weight4, x) + bias4
    return x[0][0]

In [109]:
input_test = np.array([[D, S, tau]])
print(np.shape(input_test))
input_test = np.transpose(input_test)
print(np.shape(input_test))

(1, 3)
(3, 1)


In [110]:
our_prediction = compute_prediction(input_test)
print(our_prediction)
print(pred)

12.866371869133928
12.866371154785156


Now we compute rho and eps from this (see appendix A of central paper)

In [111]:
v_star = S/(tau + D + our_prediction)
W_star = 1/np.sqrt(1-v_star**2)

rho_star = D/W_star
eps_star = (tau + D*(1 - W_star) + our_prediction*(1 - W_star**2))/(D*W_star)
print("Our calculations:")
print(rho_star, eps_star)
print("Exact results:")
print(rho, eps)

Our calculations:
9.836326155512264 1.9620391642983483
Exact results:
9.83632270803203 1.962038705851822


## (to do) Save as hdf5 file

In [112]:
# # Open an HDF5 file for writing
# with h5py.File("NNC2Pv0_params.h5", "w") as f:
#     # Save the weights and biases of the network to the HDF5 file
#     f.create_dataset("NNC2Pv0_params", data=NNC2P.state_dict())

# Using Torch script and tracing the network

There exist two ways of converting a PyTorch model to Torch Script. The first is known as tracing, a mechanism in which the structure of the model is captured by evaluating it once using example inputs, and recording the flow of those inputs through the model. This is suitable for models that make limited use of control flow. The second approach is to add explicit annotations to your model that inform the Torch Script compiler that it may directly parse and compile your model code, subject to the constraints imposed by the Torch Script language.

In [26]:
example = torch.tensor([1, 1, 0.5])
example

tensor([1.0000, 1.0000, 0.5000])

In [27]:
traced_script_module = torch.jit.trace(model, example)
traced_script_module

NeuralNetwork(
  original_name=NeuralNetwork
  (stack): Sequential(
    original_name=Sequential
    (0): Linear(original_name=Linear)
    (1): Sigmoid(original_name=Sigmoid)
    (2): Linear(original_name=Linear)
    (3): Sigmoid(original_name=Sigmoid)
    (4): Linear(original_name=Linear)
  )
)

In [28]:
output = traced_script_module(torch.tensor([1,1,0.5]))
output

tensor([0.0598], grad_fn=<AddBackward0>)

In [29]:
traced_script_module.save("NNC2Pv0t2.pt")

__To do: finish it__